In [11]:
from __future__ import print_function

import numpy as np

from openpyxl import Workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

import os

import pandas as pd

In [12]:
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.core import metrics
from evo.tools.settings import SETTINGS
from evo.tools import log
log.configure_logging()


# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

from kitti_odometry import KittiEvalOdom

In [13]:
from evo.tools import file_interface
from evo.core import sync

In [30]:
folder_with_gt_poses = '/media/cds-s/data/Datasets/Habitat/temp_gt/'
folder_with_predicted_poses = '/media/cds-s/data/Datasets/Habitat/OpenVSLAM_results/'
output_folder = '/media/cds-s/data/Datasets/Habitat/temp_result_evaluation/'
plot_mode = PlotMode.zx # 'xy', 'xz', 'yx', 'yz', 'zx', 'zy'
type_of_statistics = 'max'

In [15]:
def function_statistics(type_of_stat, list_of_values):
    if type_of_stat == 'mean':
        return np.mean(list_of_values)
    if type_of_stat == 'median':
        return np.median(list_of_values)
    if type_of_stat == 'std':
        return np.std(list_of_values)
    if type_of_stat == 'var':
        return np.var(list_of_values)
    if type_of_stat == 'min':
        return np.min(list_of_values)
    if type_of_stat == 'max':
        return np.max(list_of_values)

In [16]:
results = {}

In [24]:
def get_and_save_results_from_folder(folder_with_predicted_poses):
    
    global folder_with_gt_poses
    global results
    global output_folder
    
#     if folder.split('/')[-1] == '':
#         folder_with_predicted_poses = folder.split('/')[-2]
#     else:
#         folder_with_predicted_poses = folder.split('/')[-1]
    for filename in sorted(os.listdir(folder_with_predicted_poses)):
        if not(os.path.exists(os.path.join(folder_with_gt_poses, filename))):
            print("file with gt poses doesn't exist for "+filename)
            continue
        folder_name = folder_with_predicted_poses.split('/')[-1]
        if folder_name == '':
            folder_name = folder_with_predicted_poses.split('/')[-2]
        results[folder_name] = {
            "Kitti trans err (%)": [],
            "Kitti rot err (deg/m)": [],
            "ATE (m)": [],
            "APE(trans err) median (m)": [],
            "APE(rot err) median (deg)": []
        }
        # -------------------------------------Getting results---------------------------------------------------
        traj_ref = file_interface.read_tum_trajectory_file(os.path.join(folder_with_gt_poses, filename))
        traj_est = file_interface.read_tum_trajectory_file(os.path.join(folder_with_predicted_poses, filename))
        print(traj_ref.path_length)
        traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est)
        data = (traj_ref, traj_est)
        ape_metric_translation = metrics.APE(metrics.PoseRelation.translation_part)
        ape_metric_rotation = metrics.APE(metrics.PoseRelation.rotation_angle_deg)
        ape_metric_translation.process_data(data)
        ape_metric_rotation.process_data(data)
        ape_translation_statistics = ape_metric_translation.get_all_statistics()
        ape_rotation_statistics = ape_metric_rotation.get_all_statistics()
        kitti_trans_err, kitti_rot_err, ate = kitti_eval_tool.eval(traj_ref.poses_se3, 
                                                               traj_est.poses_se3, 
                                                               alignment=None)
    
        #---------------------------------adding results to global variable results -----------------------------
        results[folder_name]["Kitti trans err (%)"].append(kitti_trans_err)
        results[folder_name]["Kitti rot err (deg/m)"].append(kitti_rot_err)
        results[folder_name]["ATE (m)"].append(ate)
        results[folder_name]["APE(trans err) median (m)"].append(ape_translation_statistics["median"])
        results[folder_name]["APE(rot err) median (deg)"].append(ape_rotation_statistics["median"])
        #--------------------------------------------------------------------------------------------------------
        
        #----------------------------------preparing results for saving in excel--------------------------
        values_for_excel.append('{:.5f}'.format(kitti_trans_err).replace(".", ","))
        values_for_excel.append('{:.5f}'.format(kitti_rot_err).replace(".", ","))
        values_for_excel.append('{:.5f}'.format(ate).replace(".", ","))
        values_for_excel.append('{:.5f}'.format(ape_translation_statistics["median"]).replace(".", ","))
        values_for_excel.append('{:.5f}'.format(ape_rotation_statistics["median"]).replace(".", ","))
        
        columns_for_excel.append("Kitti trans err (%)")
        columns_for_excel.append("Kitti rot err (deg/m)")
        columns_for_excel.append("ATE (m)")
        columns_for_excel.append("APE(trans err) median (m)")
        columns_for_excel.append("APE(rot err) median (deg)")
        #-------------------------------------------------------------------------------------------------------    
    
        print('Results for "'+filename+'":')
        os.makedirs(os.path.join(output_folder, folder_name), exist_ok=True)
        output_folder_seq = os.path.join(output_folder, folder_name, filename[:filename.rfind('.')])
        os.makedirs(output_folder_seq, exist_ok=True)
        file_results_txt = open(os.path.join(output_folder, folder_name,"results.txt"), "w")
        file_results_txt.write("translation_error(%) rotation_error(deg/m) ATE(m) APE_translation_error_median(m) APE_rotation_error_median(deg)\n")
    
        # ------------------------------------Saving metrics to text file---------------------------------------
        txt_filename = filename[:filename.rfind('.')]+"_metrics.txt"
        with open(os.path.join(output_folder_seq, txt_filename), "w") as txt_file:
            txt_file.write('Kitti average translational error (%): {:.5f}\n'.format(kitti_trans_err))
            print('Kitti average translational error (%): {:.5f}'.format(kitti_trans_err))
            txt_file.write('Kitti average rotational error (deg/m): {:.5f}\n'.format(kitti_rot_err))
            print('Kitti average rotational error (deg/m): {:.5f}'.format(kitti_rot_err))
            txt_file.write('ATE (m): {:.5f}\n'.format(ape_translation_statistics["rmse"]))
            print('ATE (m): {:.5f}'.format(ape_translation_statistics["rmse"]))
            txt_file.write('APE(translation error) median (m): {:.5f}\n'.format(ape_translation_statistics["median"]))
            print('APE(translation error) median (m): {:.5f}'.format(ape_translation_statistics["median"]))
            txt_file.write('APE(rotation error) median (deg): {:.5f}\n'.format(ape_rotation_statistics["median"]))
            print('APE(rotation error) median (deg): {:.5f}'.format(ape_rotation_statistics["median"]))
        #---------------------------------------------------------------------------------------------------------
    
        # ---------------------------------Saving values of errors for each frame to text file------------------------
        # ------------------------------------------for translation errors----------------------------------------
        txt_filename = filename[:filename.rfind('.')]+"_APE(translation)_errors.txt"
        output_folder_seq_translation = os.path.join(output_folder_seq,"translation")
        output_folder_seq_rotation = os.path.join(output_folder_seq,"rotation")
        os.makedirs(output_folder_seq_translation, exist_ok=True)
        os.makedirs(output_folder_seq_rotation, exist_ok=True)
        with open(os.path.join(output_folder_seq_translation, txt_filename), "w") as txt_file:
            for error in ape_metric_translation.error:
                txt_file.write('{:.10f}\n'.format(error))
        # -----------------------------------------for rotation degree errors--------------------------------------
        txt_filename = filename[:filename.rfind('.')]+"_APE(rotation_deg)_errors.txt"
        with open(os.path.join(output_folder_seq_rotation, txt_filename), "w") as txt_file:
            for error in ape_metric_rotation.error:
                txt_file.write('{:.10f}\n'.format(error))
        #----------------------------------------------------------------------------------------------------------
            
        # ---------------------------------------Saving plot of errors of each frame------------------------------
        # ------------------------------------------for translation errors----------------------------------------
        plot_collection = plot.PlotCollection("Example")
        fig_1 = plt.figure(figsize=(8, 8))
        plot.error_array(fig_1, ape_metric_translation.error, 
                         name="APE", title=str(ape_metric_translation), xlabel="Index of frame", ylabel='Error')
        plot_collection.add_figure("raw", fig_1)
        plot_filename = filename[:filename.rfind('.')]+"_APE(translation)_errors.png"
        plt.savefig(os.path.join(output_folder_seq_translation, plot_filename))
        plt.close(fig_1)
        # -----------------------------------------for rotation degree errors--------------------------------------
        plot_collection = plot.PlotCollection("Example")
        fig_1 = plt.figure(figsize=(8, 8))
        plot.error_array(fig_1, ape_metric_rotation.error, 
                         name="APE", title=str(ape_metric_rotation), xlabel="Index of frame", ylabel='Error')
        plot_collection.add_figure("raw", fig_1)
        plot_filename = filename[:filename.rfind('.')]+"_APE(rotation)_errors.png"
        plt.savefig(os.path.join(output_folder_seq_rotation,plot_filename))
        plt.close(fig_1)
        #-----------------------------------------------------------------------------------------------------------
    
        # -----------------------------------------Saving trajectory plot------------------------------------------- 
        # ------------------------------------------for translation errors----------------------------------------
        fig_2 = plt.figure(figsize=(8, 8))
        ax = plot.prepare_axis(fig_2, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, '--', 'gray', 'reference')
        plot.traj_colormap( ax, traj_est, ape_metric_translation.error, plot_mode, 
                           min_map=ape_translation_statistics["min"],
                           max_map=ape_translation_statistics["max"], title="APE translation mapped onto trajectory")
        plot_collection.add_figure("traj (error)", fig_2)
        plot_filename = filename[:filename.rfind('.')]+"_APE(translation)_map.png"
        plt.savefig(os.path.join(output_folder_seq_translation,plot_filename))
        plt.close(fig_2)
        # -----------------------------------------for rotation degree errors--------------------------------------
        fig_2 = plt.figure(figsize=(8, 8))
        ax = plot.prepare_axis(fig_2, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, '--', 'gray', 'reference')
        plot.traj_colormap( ax, traj_est, ape_metric_rotation.error, plot_mode, 
                           min_map=ape_rotation_statistics["min"],
                           max_map=ape_rotation_statistics["max"], title="APE rotation mapped onto trajectory")
        plot_collection.add_figure("traj (error)", fig_2)
        plot_filename = filename[:filename.rfind('.')]+"_APE(rotation)_map.png"
        plt.savefig(os.path.join(output_folder_seq_rotation,plot_filename))
        plt.close(fig_2)
        #-----------------------------------------------------------------------------------------------------------
        print()

        file_results_txt.write('\nMean errors: {:.7f} {:.7f} {:.7f} {:.7f} {:.7f}'.format(
            np.mean(results[folder_name]["Kitti trans err (%)"]),
            np.mean(results[folder_name]["Kitti rot err (deg/m)"]),
            np.mean(results[folder_name]["ATE (m)"]),
            np.mean(results[folder_name]["APE(trans err) median (m)"]),
            np.mean(results[folder_name]["APE(rot err) median (deg)"])))
        file_results_txt.write('\nMedian errors: {:.7f} {:.7f} {:.7f} {:.7f} {:.7f}'.format(
            np.median(results[folder_name]["Kitti trans err (%)"]),
            np.median(results[folder_name]["Kitti rot err (deg/m)"]),
            np.median(results[folder_name]["ATE (m)"]),
            np.median(results[folder_name]["APE(trans err) median (m)"]),
            np.median(results[folder_name]["APE(rot err) median (deg)"])))
        file_results_txt.write('\nMax errors: {:.7f} {:.7f} {:.7f} {:.7f} {:.7f}'.format(
            np.max(results[folder_name]["Kitti trans err (%)"]),
            np.max(results[folder_name]["Kitti rot err (deg/m)"]),
            np.max(results[folder_name]["ATE (m)"]),
            np.max(results[folder_name]["APE(trans err) median (m)"]),
            np.max(results[folder_name]["APE(rot err) median (deg)"])))
        file_results_txt.write('\nMin errors: {:.7f} {:.7f} {:.7f} {:.7f} {:.7f}'.format(
            np.min(results[folder_name]["Kitti trans err (%)"]),
            np.min(results[folder_name]["Kitti rot err (deg/m)"]),
            np.min(results[folder_name]["ATE (m)"]),
            np.min(results[folder_name]["APE(trans err) median (m)"]),
            np.min(results[folder_name]["APE(rot err) median (deg)"])))
    
        file_results_txt.close()

In [25]:
plt.ioff()
kitti_eval_tool = KittiEvalOdom()
values_for_excel = []
columns_for_excel = []

In [26]:
for filename in os.listdir(folder_with_predicted_poses):
    if os.path.isfile(os.path.join(folder_with_predicted_poses, filename)):
        get_and_save_results_from_folder(folder_with_predicted_poses)
    else:
        for subdir_with_predicted_poses in os.listdir(folder_with_predicted_poses):
            print("Processing subdirectory "+subdir_with_predicted_poses)
            get_and_save_results_from_folder(os.path.join(folder_with_predicted_poses, subdir_with_predicted_poses))

Processing subdirectory 10_25_0_0
6.000000152154711
Results for "done_10_25_0_0_40909.txt":
Kitti average translational error (%): 153.02467
Kitti average rotational error (deg/m): 20.95505
ATE (m): 3.45662
APE(translation error) median (m): 3.58736
APE(rotation error) median (deg): 156.48929

Processing subdirectory 10_25_20_5
6.759267377179896
Results for "done_10_25_20_5_40909.txt":
Kitti average translational error (%): 111.68076
Kitti average rotational error (deg/m): 17.26417
ATE (m): 3.42271
APE(translation error) median (m): 3.49348
APE(rotation error) median (deg): 140.63822

Processing subdirectory 10_25_0_0
6.000000152154711
Results for "done_10_25_0_0_40909.txt":
Kitti average translational error (%): 153.02467
Kitti average rotational error (deg/m): 20.95505
ATE (m): 3.45662
APE(translation error) median (m): 3.58736
APE(rotation error) median (deg): 156.48929

Processing subdirectory 10_25_20_5
6.759267377179896
Results for "done_10_25_20_5_40909.txt":
Kitti average trans

In [23]:
results

{'10_25_0_0': {'Kitti trans err (%)': [153.02466716198873],
  'Kitti rot err (deg/m)': [20.955050850244394],
  'ATE (m)': [3.528204034059476],
  'APE(trans err) median (m)': [3.587355263192214],
  'APE(rot err) median (deg)': [156.48928716437925]},
 '10_25_20_5': {'Kitti trans err (%)': [111.6807571908787],
  'Kitti rot err (deg/m)': [17.264167542718372],
  'ATE (m)': [3.31541778783744],
  'APE(trans err) median (m)': [3.4934843263003144],
  'APE(rot err) median (deg)': [140.63822339560272]}}

In [276]:
shift_dataset = 0
shift_statitics = 1
shift_values = 5

In [277]:
wb = Workbook()
for sheet_name in wb.sheetnames:
    del wb[sheet_name]
sheet1 = wb.create_sheet('sheet1',0)
active_worksheet = wb['sheet1']
thin = Side(border_style="thin", color="000000")
thick = Side(border_style="thick", color="000000")
medium = Side(border_style="medium", color="000000")
font_header = Font(name='Arial',
                   size=10,
                   bold=True,
                   italic=False,
                   vertAlign=None,
                   underline='none',
                   strike=False,
                   color='FF000000')
font_values = Font(name='Arial',
                   size=10,
                   bold=False,
                   italic=False,
                   vertAlign=None,
                   underline='none',
                   strike=False,
                   color='FF000000')

df = pd.DataFrame.from_dict({"results":values_for_excel}, columns=columns_for_excel, orient='index')
active_worksheet.row_dimensions[2].height = 35

#-------------------------------- output names of sequences into Excel -----------------------------------------
shift = shift_statitics+shift_values+shift_dataset
for i, filename in enumerate(sorted(os.listdir(folder_with_predicted_poses))):
    active_cell = active_worksheet.cell(row = 1, column=shift+i*5+1)
    active_cell.alignment = Alignment(horizontal='center', vertical='center')
    active_cell.value = filename[:filename.rfind('.')]
    active_cell.border = Border(top=thin, left=medium, right=medium, bottom=thin)
    active_cell.font = font_header
    active_worksheet.merge_cells(start_row=1, 
                                 start_column=shift+i*5+1, 
                                 end_row=1, 
                                 end_column=shift+i*5+5)
#--------------------------------------------------------------------------------------------------------------
    
#-------------------------------- output values and names of errors into Excel ----------------------------------
shift = shift_statitics+shift_values+shift_dataset
for i in range(len(df.iloc[0,:])):
    active_worksheet.column_dimensions[get_column_letter(shift+i+1)].width = 10
    active_cell = active_worksheet.cell(row = 3+shift_dataset, column=shift+i+1)
    active_cell.value = df.iloc[0,i]
    active_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    active_cell.font = font_values
    active_worksheet.cell(row = 2, column=shift+i+1).value = columns_for_excel[i%5]
    active_worksheet.cell(row = 2, column=shift+i+1).font = font_values
    active_worksheet.cell(row = 2, column=shift+i+1).border = Border(top=thin, 
                                                                     left=thin, 
                                                                     right=thin, 
                                                                     bottom=thin)
    active_worksheet.cell(row = 2, column=shift+i+1).alignment = Alignment(horizontal='left', 
                                                                                vertical='top', 
                                                                                wrap_text=True, 
                                                                                wrapText=True)
    if i % 5 == 4:
        active_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        active_worksheet.cell(row = 2, column=shift+i+1).border = Border(top=thin, 
                                                                       left=thin, 
                                                                       right=medium, 
                                                                       bottom=thin)
#-----------------------------------------------------------------------------------------------------------------
        
#--------------------------------------------- output statistics ------------------------------------------------
shift = shift_statitics+shift_dataset
active_cell = active_worksheet.cell(row = 1, column=shift+1)
active_worksheet.merge_cells(start_row=1, start_column=shift+1, end_row=1, end_column=shift+5)
active_cell.border = Border(top=thick, left=thick, right=thick, bottom=thin)
active_cell.value = type_of_statistics + ' errors'
active_cell.font = font_header
active_cell.alignment = Alignment(horizontal='center', vertical='center')

for i in range(5):
    active_worksheet.column_dimensions[get_column_letter(shift+i+1)].width = 11
    active_cell = active_worksheet.cell(row=2, column=shift+i+1)
    active_cell.value = columns_for_excel[i]
    active_cell.font = font_header
    active_cell.alignment = Alignment(horizontal='left', vertical='top', wrap_text=True, wrapText=True)
    active_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    if i == 4:
        active_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)

map_of_number_and_error = {
    0: list_of_kitti_trans_errors,
    1: list_of_kitti_rot_errors,
    2: list_of_ATE,
    3: list_of_APE_trans_errors,
    4: list_of_APE_rot_errors
}
for i in range(5):
    active_cell = active_worksheet.cell(row=3+shift_dataset, column=shift+i+1)
    active_cell.value = '{:.5f}'.format(function_statistics(type_of_statistics, map_of_number_and_error[i]))
    active_cell.font = font_values
    active_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    if i == 4:
        active_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
    active_cell.alignment = Alignment(horizontal='left', vertical='top')
#-----------------------------------------------------------------------------------------------------------------
        
wb.save(os.path.join(output_folder,"results.xlsx"))

In [299]:
folder_with_predicted_poses

'/media/cds-s/data/Datasets/Husky-NKBVS/temp_results/'

In [ ]:
active_worksheet.iter_rows

In [31]:
print(len(next(os.walk(folder_with_predicted_poses))[1]))

6
